# Atenção este NÃO é o arquivo principal do projeto, por favor iniciem suas analises pelo arquivo principal: ArquivoPrincipal.ipnynb

___

## Preparando o ambiente

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [7]:
%%capture
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Montando o Classificador Naive-Bayes

In [8]:
produto = 'RedBull'
n = 500
t = 300

readerTrain = pd.read_excel('{0}.xlsx'.format(produto))
readerTest = pd.read_excel('{0}.xlsx'.format(produto),sheetname = 'Teste')
train = []
test = []

for i in range(0,t):
    train.append([readerTrain['Treinamento'][i], readerTrain['Util'][i]])

for i in range(0,n-t):
    test.append([readerTest['Teste'][i],readerTest['Util'][i]])
    

In [9]:
def multiply(n):
    total = 1
    for i in range(0, len(n)):
        total *= n[i]
    return total

In [10]:
class NaiveBayes:
    def __init__(self):
        self.YesFreq = 0
        self.NoFreq = 0
        self.WordFreqList = [["redbull",0,0]]
                            #[Palavra, FreqNo, FreqYes]
        self.TotalWord = [1,0,0]
                            #Palavra, FreqNo, FreqYes
        
    def train(self,train):
        for tweet in train:
            if tweet[1]: self.YesFreq+=1 #1 = Coluna Util
            else: self.NoFreq+=1
            tweet[0] = self.normalizeTweet(tweet[0])#0 = Coluna Treinamento
            WordList = tweet[0].split(' ') #0 = Coluna Treinamento
            for word in WordList:
                onList = False
                for freq in self.WordFreqList:
                    if word == freq[0]:
                        onList = True
                        if tweet[1]: 
                            freq[2]+=1 #1 = Coluna Util
                            self.TotalWord[2]+=1
                        else: 
                            freq[1]+=1
                            self.TotalWord[1]+=1
                
                if not onList:
                    self.TotalWord[0]+=1
                    if tweet[1]: self.WordFreqList.append([word,0,1]) #1 - Coluna Util
                    else: self.WordFreqList.append([word,1,0])
        return 0
    
    def test(self, test):
        acertoTotal = 0 
        erroTotal = 0
        acertoRelevante = 0
        erroRelevante = 0
        acertoIrrelevante = 0
        erroIrrelevante = 0
        for tweet in test:
            if self.analyze(tweet)==tweet[1]:
                acertoTotal+=1
                if tweet[1]: acertoRelevante+=1
                else: acertoIrrelevante+=1
            else:
                erroTotal+=1
                if tweet[1]: erroRelevante+=1
                else: erroIrrelevante+=1
        print("Positivo Falso: " + str((erroRelevante/(n-t))*100)+"%")
        print("Positivo Verdadeiro: " + str((acertoRelevante/(n-t))*100)+"%")
        print("Negativo Falso: " + str((erroIrrelevante/(n-t))*100)+"%")
        print("Negativo Verdadeiro: " + str((acertoIrrelevante/(n-t))*100)+"%")
        
        print()

        print("Acerto: " + str(((acertoIrrelevante+acertoRelevante)/(n-t))*100)+"%")
        print("Erro: "  + str(((erroIrrelevante+erroRelevante)/(n-t))*100)+"%")

        print()

        print("Total: " + str(((erroIrrelevante+erroRelevante+acertoIrrelevante+acertoRelevante)/(n-t))*100)+"%")
             
    
    def analyze(self,tweet):
        tweet[0] = self.normalizeTweet(tweet[0])#0 = Coluna Treinamento
        WordList = tweet[0].split(' ') #0 = Coluna Treinamento
        relevancia = [] #acho que nao ta sendo usado
        probWord = []
        for word in WordList:
            onList = False
            for freq in self.WordFreqList:
                if word == freq[0]:
                    onList = True
                    probNo = (freq[1]+1)/(self.TotalWord[1]+self.TotalWord[0])
                    probYes = (freq[2]+1)/(self.TotalWord[2]+self.TotalWord[0])
                    probTotal = (freq[1]+freq[2]+1)/self.TotalWord[0]
                    probWord.append([probTotal,probNo,probYes])
                    
            if not onList:
                probWord.append([1/self.TotalWord[0],1/self.TotalWord[1],1/self.TotalWord[2]])
        
        probTweet = (multiply(probWord[:][1]),multiply(probWord[:][2]))
        if((probTweet[0]>probTweet[1])): return 1
        else: return 0
 
    def normalizeTweet(self,tweet):
        tweet.lower()
        tweet = tweet.replace(',',' ')
        tweet = tweet.replace('!',' ')
        tweet = tweet.replace('$',' ')
        tweet = tweet.replace('%',' ')
        tweet = tweet.replace('&',' ')
        tweet = tweet.replace('*',' ')
        tweet = tweet.replace('(',' ')
        tweet = tweet.replace(')',' ')
        tweet = tweet.replace('-',' ')
        tweet = tweet.replace('+',' ')
        tweet = tweet.replace('=',' ')
        tweet = tweet.replace('_',' ')
        tweet = tweet.replace('.',' ')
        tweet = tweet.replace(':',' ')
        tweet = tweet.replace(';',' ')
        tweet = tweet.replace('?',' ')
        tweet = tweet.replace('/',' ')
        tweet = tweet.replace('\n',' ')
        for i in range(0,4): tweet = tweet.replace('  ',' ')
        return tweet
                              
    def show(self):
        print("SIMs: " + str(self.YesFreq))
        print("NÃOs: " + str(self.NoFreq))
        print("TOTAL: " + str(self.YesFreq+self.NoFreq))
    
    def showAll(self):
        print(self.WordFreqList)
        print()
        print("SIMs: " + str(self.YesFreq))
        print("NÃOs: " + str(self.NoFreq))
        print("TOTAL: " + str(self.YesFreq+self.NoFreq))
        


In [11]:
Classificador = NaiveBayes()
Classificador.train(train)
Classificador.test(test)

Positivo Falso: 2.5%
Positivo Verdadeiro: 3.5000000000000004%
Negativo Falso: 12.5%
Negativo Verdadeiro: 81.5%

Acerto: 85.0%
Erro: 15.0%

Total: 100.0%


Assim como dito no arquivo principal, o Classficador classificou com maior frequencia a categoria que teve maior incidencia durante a banda de treinos, porem aqui isso mostrou um resultado bem melhor